In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import SVD
# import accuracy

import warnings; warnings.simplefilter('ignore')

In [2]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [4]:
import json
import pandas as pd
import numpy as np

import requests

from tqdm import tqdm_notebook
from collections import Counter

# 성인 - 책 소개 기반 추천

- 데이터 불러오기

In [23]:
adult_final_df=pd.read_excel('./data/final/content_re/adult_final_df.xlsx')

In [24]:
adult_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                888 non-null    int64  
 1   ranking           888 non-null    int64  
 2   bookname          888 non-null    object 
 3   authors           888 non-null    object 
 4   publisher         883 non-null    object 
 5   publication_year  875 non-null    float64
 6   isbn13            888 non-null    int64  
 7   addition_symbol   872 non-null    float64
 8   vol               84 non-null     float64
 9   class_no          876 non-null    float64
 10  loan_count        888 non-null    int64  
 11  bookImageURL      888 non-null    object 
 12  keyword           842 non-null    object 
 13  describe          888 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 97.2+ KB


In [25]:
adult_final_df

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL,keyword,describe
0,116,116,존리의 부자되기 습관 :대한민국 경제독립 액션 플랜,존리 지음,지식노마드,2020.0,9791187481720,13320.0,NaN,327.04,1371,https://image.aladin.co.kr/product/22631/95/co...,"['펀드', '경제독립', '투자', '금융문맹', '부자', '시작', '한국',...",돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1,469,469,당신이 옳다 :정혜신의 적정심리학,정혜신,NaN,NaN,9788965746669,NaN,NaN,189.00,684,http://image.kyobobook.co.kr/images/book/large...,"['공감', '심리', '치유', '심리치유', '콤플렉스', '내공']",안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2,648,648,남아 있는 시간을 위하여:100세 철학자의 대표산문선,김형석,NaN,NaN,9788934980629,NaN,NaN,104.00,565,http://image.kyobobook.co.kr/images/book/large...,NaN,인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...
3,663,660,인생 우화,류시화,NaN,NaN,9791186686348,NaN,NaN,813.80,556,http://image.kyobobook.co.kr/images/book/large...,NaN,우리 안의 바보는 어떤 엉뚱한 방식으로 문제를 해결하며 살아갈까?\n류시화 시인이 ...
4,699,698,인생 수업,"엘리자베스 퀴블러 로스,류시화 옮김",이레,2006.0,9788957090817,3840.0,NaN,199.10,535,https://bookthumb-phinf.pstatic.net/cover/024/...,"['자신', '마지막', '바쇼하이쿠선집', '삶으로다시떠오르기', '사랑하라한번도...",배움을 얻는다는 것은 다른 사람이 아닌 자기 자신의 인생을 사는 것을 의미합니다. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,995,990,수도권 알짜 부동산 답사기 :알짜 노하우,김학렬 지음,지혜로,2019.0,9791187799108,13320.0,NaN,327.87,445,https://image.aladin.co.kr/product/19916/14/co...,"['서울', '레시피', '동네', '부동산', '지역분석', '인천', '지역',...",대한민국 역사상 가장 강력한 부동산 규제정책이 펼쳐지고 있다. 내 집 마련을 해야 ...
884,996,990,사라진 세계,톰 스웨터리치 지음 ;장호연 옮김,허블,2020.0,9791190090070,3840.0,NaN,843.60,445,https://image.aladin.co.kr/product/23332/89/co...,NaN,"허리우드가 주목하는, 특히, 아카데미 후보작 《디스트릭트9》으로 세계적인 명성을 쌓..."
885,997,990,나를 아프게 하지 않는다 :상처만 주는 가짜 자존감 나를 지키는 진짜 자존감,전미경 지음,지와인,2019.0,9791196533472,3180.0,NaN,189.10,445,https://image.aladin.co.kr/product/21831/77/co...,"['자존감', '사람', '자기', '감정', '가짜', '관계', '시그니처', ...",우리는 어쩌다 자존감의 시대를 살게 되었을까? 원래 자존감이라는 게 이렇게 만들어지...
886,998,998,김미경의 인생미답 :살다 보면 누구나 마주하는 작고 소소한 질문들,김미경 지음,한경BP,2016.0,9788947540926,3320.0,NaN,199.10,444,http://image.aladin.co.kr/product/8193/51/cove...,"['김미경', '자신', '문제', '인생', '사랑', '소소', '질문', '김...",스타 강사 김미경은 오랫동안 데리고 살아온 고민과 나름의 생각을 모두와 공유하고 싶...


In [26]:
# 책소개가 없는 데이터지우기
adult_final_df=adult_final_df[pd.notnull(adult_final_df['describe'])]

In [27]:
# 인덱스 정리
adult_final_df.reset_index(drop=True,inplace=True)

In [33]:
adult_final_df['bookname'][4]

'인생 수업'

- 책소개를 형용사+명사 데이터로 바꿈

In [46]:
book_intro=[]
for i in range(len(adult_final_df)):
    sentences_tag = []
    lines=adult_final_df['describe'][i].replace('\n',' ').split('.')
    twitter = Twitter()
    for sentence in lines:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                noun_adj_list.append(word)
    book_intro.append(noun_adj_list)

In [47]:
len(book_intro)

888

In [48]:
adult_final_df['book_intro']=book_intro

In [54]:
intro=[]
for i in range(len(adult_final_df)):
    s=' '.join(adult_final_df['book_intro'][i])
    intro.append(s)

In [55]:
adult_final_df['intro']=intro

In [57]:
adult_final_df['describe'].head(15)

0     돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1     안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2     인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...
3     우리 안의 바보는 어떤 엉뚱한 방식으로 문제를 해결하며 살아갈까?\n류시화 시인이 ...
4     배움을 얻는다는 것은 다른 사람이 아닌 자기 자신의 인생을 사는 것을 의미합니다. ...
5     극한의 제약, 무한의 상상력,\n히가시노 게이고만이 가능한 고밀도 미스터리!\n\n...
6     당신은 행복해질 권리가 있다\n법륜 스님은 지난 30년 동안 우리나라 전국방방곡곡은...
7     있는 그대로 참 소중한 당신에게 전하는 진심 가득한 위로의 말\n2년이 넘도록 전국...
8     자유인으로 돌아온 유시민, '어떻게 살 것인지'에 대해 되짚어본다!\n정치인에서 자...
9     바로 지금, 자신에게 맞는 재미를 찾는 것이 진정 나이답게 늙어가는 일이다!\n멋지...
10    여행의 감각을 일깨우는 소설가 김영하의 매혹적인 이야기 『여행의 이유』. 꽤 오래전...
11    거대 자본에 휘둘려 인간을 소외시킨 현 상황을 통찰해 대한민국의 미래를 재편하는 조...
12    은밀하고 사소하며 일상적이고 자연스럽게 벌어지는 일들 속에서 선량한 우리가 놓치고 ...
13    바이오센서를 만드는 과학도에서 이제는 소설을 쓰는 작가 김초엽. 어디에도 없는 그러...
14    2012년 12월 19일 국내 독자들과 처음 만났던 『나미야 잡화점의 기적』이 출간...
Name: describe, dtype: object

In [56]:
adult_final_df['intro'].head(15)

0     돈 위해 일 돈 당신 위해 일 하라 일 왜 돈 삶 살 있는가 한국인 세계 나라 머리...
1     안정 일상 위해 스스로 마음 문제 해결 수 있도록 공감 행동지침 여 년 정신과 의사...
2     인생 마지막 더욱 간절해지는 선하고 아름다운 삶 향 고민 소산 고독 병 영원 사랑 ...
3     우리 안 바보 어떤 엉뚱한 방식 문제 해결 류시화 시인 우화 인생 우화 폴란드 전해...
4     배움 것 다른 사람 아닌 자기 자신 인생 것 의미 갑자기 더 행복해지거나 부자 강해...
5     극한 제약 무한 상상력 게이고 가능한 밀도 미스터리 저 야심작 자신 있게 추천 게이...
6     당신 행복해질 권리 있다 법륜 스님 지난 동안 우리나라 전국 방방곡곡 세계 개 도시...
7     있는 그대로 소중한 당신 전하 진심 가득한 위로 말 전국 서점 베스트셀러 스 테디 ...
8     자유 유시민 어떻게 살 것 대해 정치인 자유 유시민 어떻게 살 것 세상 변화 누구 ...
9     바로 지금 자신 재미 것 진정 일이 멋지게 나이 사람 위 인생 기술 나 죽 때 재미...
10    여행 감각 일 소설가 김영하 매혹 이야기 여행 이유 꽤 부터 여행 대해 저자 처음 ...
11    거대 자본 휘 인간 소외 현 상황 통찰 대한민국 미래 재 편하는 조정래 장편소설 천...
12    은밀 사소하며 일상 자연 일 속 선량한 우리 있던 차별 혐오 순간 날카롭게 포착 선...
13    바이오 센서 과학 도 이제 소설 작가 김 초엽 어디 없는 어딘가 있을 것 같은 상상...
14    국내 독자 처음 나미 잡화 점 기적 이 간 만인 쇄 발행 현대문학 쇄 기념 땡큐 에...
Name: intro, dtype: object

# TF-IDF

In [58]:
# 블용어 txt파일 리스트 가져오기
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [59]:
stop_words=list(sw[0].dropna())

In [60]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(adult_final_df['intro'])

In [61]:
tfidf_matrix

<888x87210 sparse matrix of type '<class 'numpy.float64'>'
	with 150582 stored elements in Compressed Sparse Row format>

In [62]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [63]:
titles = adult_final_df['bookname']
indices = pd.Series(adult_final_df.index, index=adult_final_df['bookname'])

In [65]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [66]:
# ex1
get_recommendations('존리의 부자되기 습관 :대한민국 경제독립 액션 플랜').head(10)

738      월급쟁이 부자로 은퇴하라 :3년 만에 30년 치 연봉을 번 김 과장의 시스템 마련법 
402                         부자언니 부자연습 :가난한 공주 부자되기 프로젝트 
395                                  나는 오늘도 경제적 자유를 꿈꾼다 
629                     부자 아빠 가난한 아빠:부자들이 들려주는 돈과 투자의 비밀
287                          부자언니 부자특강 :평범한 월급쟁이 부자되는 공식
764        난생처음 주식투자 :초보도 손실 없이 5할 타율 유지하는 안전한 주식투자 교과서 
535                       마법의 돈 굴리기 :자산배분을 이용한 스노우볼 투자법 
841    (개인투자보다 안전하고 외국인과 기관투자보다 승률 높은) ETF 투자 실전 가이드북...
166    나의 월급 독립 프로젝트 :3년 만에 30억 벌고 퇴사한 슈퍼개미의 실전 주식투자 생중계
776    (짐 로저스) 앞으로 5년 한반도 투자 시나리오 =경제통합 한반도를 바라보는 월스트...
Name: bookname, dtype: object

In [67]:
# ex2
get_recommendations('나는 죽을 때까지 재미있게 살고 싶다').head(10)

702    나이든 나와 살아가는 법 :흔들리지 않고 의연하게 나이 들 수 있는 후반생의 마음 사전 
427                           인생 수업 :잘 물든 단풍은 봄꽃보다 아름답다 
314               나는 내 나이가 참 좋다 :우아하고 지혜롭게 세월의 강을 항해하는 법
289                       (어차피 살 거라면) 백 살까지 유쾌하게 나이 드는 법
53                             하마터면 열심히 살 뻔했다 :야매 득도 에세이
339                                    어른답게 삽시다 :이시형 에세이
220                            백년을 살아보니 :인생의 황금기는 60~75세
50     50부터는 인생관을 바꿔야 산다 :이제 자존심, 꿈, 사람은 버리고 오직 나를 위해서만!
2                          남아 있는 시간을 위하여:100세 철학자의 대표산문선
504          50 이후, 인생을 결정하는 열 가지 힘 - 하버드 심리학 거장의 마지막 강의
Name: bookname, dtype: object

In [68]:
get_recommendations('아주 작은 습관의 힘 :최고의 변화는 어떻게 만들어지는가').head(10)

236         해빗 =내 안의 충동을 이겨내는 습관 설계의 법칙 /Habit
312                          나는 습관을 조금 바꾸기로 했다
858              (흔들리지 않고 끝까지 계속하게 만드는) 루틴의 힘 
528                습관의 힘 :반복되는 행동이 만드는 극적인 변화 
465        예술하는 습관 :위대한 창조의 순간을 만든 구체적 하루의 기록 
332                1만 시간의 재발견 :노력은 왜 우리를 배신하는가
61                        12가지 인생의 법칙 :혼돈의 해독제
188         미라클모닝 :당신의 하루를 바꾸는 기적 아침 6분이면 충분하다
402               부자언니 부자연습 :가난한 공주 부자되기 프로젝트 
769    지금까지 산 것처럼 앞으로도 살 건가요? :내 인생의 판을 바꿀 질문 
Name: bookname, dtype: object

In [ ]:
# csv 생성
adult_final_df.to_csv('./data/final/content_re/adult_final.csv',encoding='utf-8-sig',index=False)